In [1]:
!pip install deap

     ------------------------------------ 109.3/109.3 kB 421.4 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import warnings
warnings.filterwarnings("ignore")

# Ejercicio de la mochila

In [4]:
#Definir los pesos y beneficios como listas de python
beneficios = [50,35,45,20,35,40,50,50,45,40]
pesos = [20,25,20,20,35,40,35,30,35,25]
capacidad = 120

In [5]:
#Declarar el contenedor de operadores geneticos
toolbox = base.Toolbox()


In [7]:
#Crear la clase funcion de adaptabilidad y la clase individuo
creator.create("FitnessMax", base.Fitness, weights = (1.0, ))
creator.create("Individual", list, fitness = creator.FitnessMax)


In [9]:
# Construir los individuos
# Determinar el comportamiento del gen
toolbox.register("attribute", random.randint, 0,1)
# Determinar el tamaño del individuo
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n = 10)
#Determinar la forma de la poblacion
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# Confirmar individuo y poblacion

In [10]:
ind = toolbox.individual()
print("Individuo: ", ind)
pop = toolbox.population(n=6)
print("Poblacion: ", pop)

Individuo:  [1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
Poblacion:  [[1, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 1, 1, 0, 0, 1, 1, 0], [0, 0, 1, 1, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 0, 1, 1, 1, 0, 1, 0, 0]]


# Crear la funcion de adaptabilidad

In [11]:
def evaluarBeneficio(individual):
    beneficio = 0.0
    peso = 0.0
    beneficio = np.dot(individual, beneficios)
    peso = np.dot(individual, pesos)

    if peso > capacidad:
        beneficio = beneficio - peso
    else:
        beneficio
    return beneficio,

# Confirmar la funcion de adaptabilidad


In [12]:
ind = toolbox.individual()
print("Individuo previo a evaluar")
print(ind)
print(ind.fitness.valid)
ind.fitness.values = evaluarBeneficio(ind)
print("Individuo evaluado")
print(ind)
print(ind.fitness.valid)
print(ind.fitness)


Individuo previo a evaluar
[0, 0, 1, 1, 1, 1, 0, 0, 0, 1]
False
Individuo evaluado
[0, 0, 1, 1, 1, 1, 0, 0, 0, 1]
True
(40.0,)
